<a href="https://colab.research.google.com/github/sherkhalil/COVID19/blob/master/sentiment140_bi_lstm.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#path = '/content/drive/'
#p = path + 'My Drive/Thesis New Data/Data from Dr. Anwar Ali Yahya'
dataset = '/content/drive/My Drive/NLP/twitter_sentiments.csv'

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [3]:
import pandas as pd 
data = pd.read_csv(dataset,encoding='ISO-8859-1',names=["Label", "COL2", "COL3", "COL4","COL5","Tweet"]) 
data.head()

,Label,COL2,COL3,COL4,COL5,Tweet
0,0,1467810369,Mon Apr 06 22:19:45 PDT 2009,NO_QUERY,_TheSpecialOne_,"@switchfoot http://twitpic.com/2y1zl - Awww, t..."
1,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...
2,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...
3,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire
4,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all...."


In [4]:
data.Label.value_counts()

4    800000
0    800000
Name: Label, dtype: int64

In [5]:
data['Label_Numeric'] = data['Label'].map({0:0, 4:1})

In [6]:
data['Label_Numeric'].head()
data.Label_Numeric.value_counts()


1    800000
0    800000
Name: Label_Numeric, dtype: int64

In [7]:
%tensorflow_version 1.x



from keras import backend as K
#K.tensorflow_backend._get_available_gpus()


from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
import numpy as np
maxlen = 280
max_words = 40000
tokenizer = Tokenizer(num_words=max_words)
#tokenizer.fit_on_texts(data['Tweet'])
#sequences = tokenizer.texts_to_sequences(data['Tweet'])

tokenizer.fit_on_texts(data['Tweet'])
sequences = tokenizer.texts_to_sequences(data['Tweet'])

word_index = tokenizer.word_index
print('Found %s unique tokens.' % len(word_index))
data1 = pad_sequences(sequences, maxlen=maxlen)
print(data1)
def to_one_hot(labels, dimension=2):
    results = np.zeros((len(labels), dimension))
    for i, label in enumerate(labels):
        results[i, label] = 1.
    return results
one_hot_labels = to_one_hot(data['Label_Numeric'])
labels = np.asarray(one_hot_labels)
print('Shape of data tensor:', data.shape)
print('Shape of label tensor:', labels.shape)
indices = np.arange(data1.shape[0])
np.random.shuffle(indices)
data1 = data1[indices]
labels = labels[indices]
"""x_train = data1[:training_samples]
y_train = labels[:training_samples]
x_val = data1[training_samples: training_samples + validation_samples]
y_val = labels[training_samples: training_samples + validation_samples]
x_test = data1[validation_samples: validation_samples + test_samples]
y_test = labels[validation_samples: validation_samples + test_samples]"""

#from sklearn.model_selection import train_test_split

from sklearn.model_selection import train_test_split


X_train, X_test, y_train, y_test = train_test_split(data1, labels, test_size=0.1, random_state=0)
#X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.03, random_state=0)

TensorFlow 1.x selected.


Using TensorFlow backend.


Found 690960 unique tokens.
[[    0     0     0 ...    41     9   385]
 [    0     0     0 ...    40   273  1170]
 [    0     0     0 ...    31    12 27341]
 ...
 [    0     0     0 ...    14    11  2107]
 [    0     0     0 ...  5325    50 13870]
 [    0     0     0 ...     0   119 20040]]
Shape of data tensor: (1600000, 7)
Shape of label tensor: (1600000, 2)


In [8]:
print(labels.shape)
print(y_test.shape)

(1600000, 2)
(160000, 2)


In [9]:
y_train.shape

(1440000, 2)

In [10]:
#without pretrained

from keras.models import Sequential
from keras.layers import Embedding, Flatten, Dense, Dropout
from keras.layers import Embedding, LSTM
from keras import regularizers
from keras import layers
from keras.layers import Bidirectional
from keras.layers import TimeDistributed
embedding_dim = 300

model2 = Sequential()
model2.add(Embedding(max_words, embedding_dim, input_length=maxlen))
model2.add(Bidirectional(layers.LSTM(32,return_sequences = True, dropout=0.2,recurrent_dropout=0.2)))
model2.add(Bidirectional(layers.LSTM(32, return_sequences= True)))
model2.add(layers.GlobalMaxPooling1D())
model2.add(layers.Dense(10, activation='relu'))
model2.add(layers.Dense(2, activation='sigmoid'))
model2.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])
model2.summary()



history = model2.fit(X_train, y_train, epochs=10, batch_size=2048, validation_split = 0.1, shuffle=True)



Instructions for updating:
If using Keras pass *_constraint arguments to layers.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 280, 300)          12000000  
_________________________________________________________________
bidirectional_1 (Bidirection (None, 280, 64)           85248     
_________________________________________________________________
bidirectional_2 (Bidirection (None, 280, 64)           24832     
_________________________________________________________________
global_max_pooling1d_1 (Glob (None, 64)                0         
_________________________________________________________________
dense_1 (Dense)              (None, 10)                650       
_________________________________________________________________
dense_2 (De

In [11]:
from sklearn.metrics import precision_score , recall_score
y_val_pred=model2.predict_classes(X_test)


In [12]:

from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score
from sklearn.metrics import cohen_kappa_score
from sklearn.metrics import roc_auc_score
from sklearn.metrics import confusion_matrix


rounded_labels=np.argmax(y_test, axis=1)


# accuracy: (tp + tn) / (p + n)
accuracy = accuracy_score(rounded_labels, y_val_pred)
print('Accuracy: %f' % accuracy)
# precision tp / (tp + fp)
precision = precision_score(rounded_labels, y_val_pred)
print('Precision: %f' %precision)
# recall: tp / (tp + fn)
recall = recall_score(rounded_labels, y_val_pred, average = 'weighted')
print('Recall: %f' %recall)
# f1: 2 tp / (2 tp + fp + fn)
f1 = f1_score(rounded_labels, y_val_pred)
print('F1: %f' %f1)
# accuracy: (tp + tn) / (p + n)

Accuracy: 0.816444
Precision: 0.828892
Recall: 0.816444
F1: 0.812885
